<a href="https://colab.research.google.com/github/Raziboy454/Neural-Network-SMS-Text-Classifier/blob/main/fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
# Cell 3: Data Preparation and Tokenization

# Load the training and validation data
train_df = pd.read_csv(train_file_path, sep='\t', header=None, names=['label', 'message'])
test_df = pd.read_csv(test_file_path, sep='\t', header=None, names=['label', 'message'])

# Convert labels to 0 and 1
train_df['label'] = train_df['label'].map({'ham': 0, 'spam': 1})
test_df['label'] = test_df['label'].map({'ham': 0, 'spam': 1})

# Split features and labels
X_train = train_df['message']
y_train = train_df['label']
X_test = test_df['message']
y_test = test_df['label']

# --- Improved Tokenization Parameters ---
# Use a much larger vocab size to minimize OOV words
vocab_size = 10000
max_length = 120 # Increased max length slightly, though 100 is often enough
trunc_type = 'post'
padding_type = 'post'
oov_tok = "<OOV>"

# Initialize and fit tokenizer on training data
tokenizer = keras.preprocessing.text.Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(X_train)

# Convert text to padded sequences
train_sequences = tokenizer.texts_to_sequences(X_train)
train_padded = keras.preprocessing.sequence.pad_sequences(train_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

test_sequences = tokenizer.texts_to_sequences(X_test)
test_padded = keras.preprocessing.sequence.pad_sequences(test_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

# Convert to NumPy arrays for Keras compatibility
train_padded = np.array(train_padded)
y_train = np.array(y_train)
test_padded = np.array(test_padded)
y_test = np.array(y_test)

In [ ]:
# Cell 4: Model Building and Training

embedding_dim = 32 # Increased embedding dimension
lstm_units = 32

# Build the neural network with a Bidirectional LSTM layer
model = keras.Sequential([
    keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    # Bidirectional LSTM often performs better on text classification tasks
    keras.layers.Bidirectional(keras.layers.LSTM(lstm_units)),
    keras.layers.Dense(24, activation='relu'),
    keras.layers.Dropout(0.2), # Added dropout for regularization
    keras.layers.Dense(1, activation='sigmoid') # Final output layer for binary classification
])

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
num_epochs = 15 # Increased epochs to allow better convergence
# Use callbacks for early stopping to prevent overfitting
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

history = model.fit(
    train_padded,
    y_train,
    epochs=num_epochs,
    validation_data=(test_padded, y_test),
    verbose=2,
    callbacks=[early_stop] # Added early stopping
)

# Check accuracy (optional)
loss, acc = model.evaluate(test_padded, y_test, verbose=0)
print(f"Test Accuracy: {acc:.4f}")

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  """
  Predicts whether a given text message is 'ham' or 'spam'.
  Returns a list: [likelihood of spam (float), 'ham' or 'spam' (string)].
  """
  # Convert message to sequence using the global tokenizer
  seq = tokenizer.texts_to_sequences([pred_text])

  # Pad the sequence using the global max_length and padding parameters
  padded = keras.preprocessing.sequence.pad_sequences(
      seq,
      maxlen=max_length,
      padding=padding_type,
      truncating=trunc_type
  )

  # Predict probability (spam likelihood)
  # model.predict returns a 2D array, e.g., [[0.00831883]] -> extract the scalar value
  raw_prediction = model.predict(padded, verbose=0)[0][0]

  # Determine label based on a threshold (0.5 is standard for sigmoid)
  label = 'spam' if raw_prediction > 0.5 else 'ham'

  # Return the required format: [prediction_float, label_string]
  return [float(raw_prediction), label]

pred_text = "how are you doing today?"
prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
